In [3]:
import numba as nb

In [4]:
import squidpy as sq
import scanpy as sc
import pandas as pd
import numpy as np
from scipy.sparse import csr_matrix, isspmatrix_csr

In [5]:
import liana as li

In [6]:
from matplotlib.pyplot import hist

In [7]:
from liana.method.sp._spatial_pipe import _global_lr_pipe
from liana.method.sp._spatialdm import _get_ordered_matrix, _standardize_matrix

In [8]:
from liana.method.sp._bivariate_funs import _masked_coexpressions, _vectorized_correlations, _vectorized_wcosine, _vectorized_jaccard, _masked_spearman, _masked_pearson

In [9]:
from liana.method.sp._spatialdm import _local_morans, _standardize_matrix

In [ ]:
# # scHOT data
# counts = pd.read_csv("data/counts_mat.csv")
# weights = pd.read_csv("data/weight_mat.csv")
# var = pd.DataFrame(counts[['Unnamed: 0']]).set_index('Unnamed: 0')
# var.index.name = None
# adata = sc.AnnData(X=csr_matrix(counts.drop(counts.columns[0], axis=1), dtype=np.float32).T, var=var)
# adata.obsm['proximity'] = csr_matrix(weights)

In [10]:
# scHOT data test
adata = sc.read_h5ad("test_spatial.h5ad")
resource = pd.DataFrame({"ligand":["Dnm1l", "Arrb1", "Igf2", "Dnm1l"], "receptor":["Gucy1b3", "Mtor", "Tuba1a", "Fam63b"]})
dist = adata.obsm['proximity']

In [11]:
n_perm = 100
seed = 0

In [12]:
# full visium slide
# load the pre-processed dataset
img = sq.datasets.visium_hne_image()
adata = sq.datasets.visium_hne_adata()

li.mt.get_spatial_proximity(adata=adata, parameter=200, bypass_diagonal=False, cutoff=0.1, family="exponential")
dist = adata.obsm['proximity']

my_p = li.pl.proximity_plot(adata, idx=100)
resource = li.resource.select_resource("mouseconsensus")

In [13]:
temp, lr_res, ligand_pos, receptor_pos = _global_lr_pipe(adata=adata,
                                                         resource=resource,
                                                         expr_prop=0.05,
                                                         use_raw=False,
                                                         verbose=True,
                                                         layer=None,
                                                         _key_cols=['ligand_complex', 'receptor_complex'],
                                                         _complex_cols=['ligand_means', 'receptor_means'],
                                                         _obms_keys=['proximity'],
                                                         resource_name=None
                                                         )

Using `.X`!


In [14]:
# lr_res = lr_res.head(50)

In [15]:
x_mat = _get_ordered_matrix(temp.X, ligand_pos, lr_res.ligand).A.astype(np.float32).T
y_mat = _get_ordered_matrix(temp.X, receptor_pos, lr_res.receptor).A.astype(np.float32).T

In [16]:
lr_res.head()

,interaction,ligand,receptor,ligand_complex,receptor_complex,ligand_means,ligand_props,receptor_means,receptor_props,prop_min
0,Dll1&Notch1,Dll1,Notch1,Dll1,Notch1,0.037283,0.057292,0.216689,0.276786,0.057292
1,Adam10&Notch1,Adam10,Notch1,Adam10,Notch1,0.623617,0.651786,0.216689,0.276786,0.276786
2,Jag1&Notch1,Jag1,Notch1,Jag1,Notch1,0.051299,0.075893,0.216689,0.276786,0.075893
3,Adam17&Notch1,Adam17,Notch1,Adam17,Notch1,0.121026,0.167783,0.216689,0.276786,0.167783
4,Dlk1&Notch1,Dlk1,Notch1,Dlk1,Notch1,0.236641,0.213542,0.216689,0.276786,0.213542


In [17]:
dist = adata.obsm['proximity']
weight = dist.A.astype(np.float32)

In [19]:
%%time
masked_pc = _masked_pearson(x_mat, y_mat, weight, weight_thr=0.0)

CPU times: user 1min 7s, sys: 97 ms, total: 1min 7s
Wall time: 6.57 s


In [20]:
%%time
masked_sp = _masked_spearman(x_mat, y_mat, weight, weight_thr=0.0)

CPU times: user 3min 9s, sys: 418 ms, total: 3min 10s
Wall time: 17.4 s


In [21]:
masked_pc

array([[ 0.08149756, -0.25916123, -0.14290628, ..., -0.10626076,
         0.08216916,  0.        ],
       [ 0.        ,  0.07690792, -0.07125933, ..., -0.30776626,
        -0.1647764 ,  0.19228509],
       [ 0.03793216, -0.08338802, -0.27273718, ..., -0.34547132,
         0.15676172,  0.08250511],
       ...,
       [ 0.26313928,  0.32310468, -0.22683044, ...,  0.21042919,
        -0.1380716 ,  0.19210997],
       [ 0.0876326 , -0.18898553,  0.13564353, ..., -0.01432045,
         0.6279468 ,  0.        ],
       [ 0.        , -0.01092029,  0.48072618, ...,  0.20706737,
        -0.00277106,  0.        ]], dtype=float32)

In [22]:
masked_sp

array([[ 0.18781684, -0.11848656,  0.12627968, ...,  0.34751382,
         0.24387598,  0.1599317 ],
       [ 0.30682617, -0.2751629 ,  0.29463288, ...,  0.507999  ,
         0.39533183, -0.1267864 ],
       [ 0.33031335, -0.09867558,  0.13571519, ..., -0.41641706,
         0.2175387 ,  0.04966684],
       ...,
       [ 0.31701288,  0.25899726, -0.01386184, ...,  0.37027845,
        -0.32945094,  0.6252148 ],
       [ 0.43997544,  0.0042903 ,  0.4969458 , ..., -0.04243322,
         0.517518  ,  0.0436135 ],
       [ 0.42425486,  0.01879715,  0.55392396, ...,  0.26318154,
         0.3640542 ,  0.43966317]], dtype=float32)

In [ ]:
_masked_pearson(x_mat, y_mat, weight, 0.0)

Fully-vectorized

In [ ]:
import squidpy as sq
import scanpy as sc
import pandas as pd
import numpy as np
from scipy.sparse import csr_matrix, isspmatrix_csr

In [ ]:
import liana as li

In [ ]:
from matplotlib.pyplot import hist
from scipy.stats import spearmanr, pearsonr

In [ ]:
from liana.method._global_lr_pipe import _global_lr_pipe
from liana.method.sp._spatialdm import _get_ordered_matrix, _standardize_matrix

In [ ]:
# ligand-receptor mats
ligand_mat = _get_ordered_matrix(temp.X, ligand_pos, lr_res.ligand).T
receptor_mat = _get_ordered_matrix(temp.X, receptor_pos, lr_res.receptor).T

In [ ]:
dist = adata.obsm['proximity']

In [ ]:
local_pc = _vectorized_correlations(x_mat = ligand_mat.A, y_mat=receptor_mat.A, dist=dist, method="pearson")

In [ ]:
local_sp = _vectorized_correlations(x_mat = ligand_mat.A, y_mat=receptor_mat.A, dist=dist, method="spearman")

In [ ]:
local_cos = _vectorized_wcosine(x_mat = ligand_mat.A, y_mat=receptor_mat.A, dist=dist)

In [ ]:
local_jc = _vectorized_jaccard(x_mat = ligand_mat.A, y_mat=receptor_mat.A, dist=dist)

In [ ]:
ligand_mat = _get_ordered_matrix(temp.X, ligand_pos, lr_res.ligand).T
receptor_mat = _get_ordered_matrix(temp.X, receptor_pos, lr_res.receptor).T

In [ ]:
# if spatial DM: standardize & calculate norm factor
ligand_mat = _standardize_matrix(ligand_mat, local=True)
receptor_mat = _standardize_matrix(receptor_mat, local=True)

In [ ]:
norm_factor = temp.obsm['proximity'].shape[0] / temp.obsm['proximity'].sum()
dist = csr_matrix(norm_factor * temp.obsm['proximity'])

In [ ]:
local_r = _local_morans(x_mat = ligand_mat, y_mat=receptor_mat, dist=dist)

In [ ]:
local_r

In [ ]:
local_r

In [ ]:
from liana.method.sp._spatial_utils import _local_permutation_pvals

In [ ]:
%%time
local_pc_pvals = _local_permutation_pvals(x_mat = ligand_mat.A, 
                                          y_mat = receptor_mat.A, 
                                          local_truth=local_pc,
                                          local_fun=_vectorized_correlations,
                                          dist=dist, 
                                          n_perm=n_perm, 
                                          positive_only=False,
                                          seed=seed)

In [ ]:
local_pc_pvals

In [ ]:
local_sp = _vectorized_correlations(x_mat = ligand_mat.A, y_mat=receptor_mat.A, dist=dist, method="spearman")

In [ ]:
local_sp_pvals = _local_permutation_pvals(x_mat = ligand_mat.A, 
                                          y_mat = receptor_mat.A, 
                                          local_truth=local_sp,
                                          local_fun=_vectorized_correlations,
                                          dist=dist, 
                                          n_perm=n_perm, 
                                          positive_only=False,
                                          seed=seed,
                                          method="spearman"
                                          )

In [ ]:
local_sp_pvals.shape

In [ ]:
spearmanr(local_sp_pvals[1,:], local_pc_pvals[1,:])

Global summary of the local scores:

In [ ]:
lr_res.loc[:,['pearson_mean','pearson_sd']] = np.vstack([np.mean(local_pc, axis=1), np.std(local_pc, axis=1)]).T

In [ ]:
lr_res.sort_values(by='pearson_mean', ascending=False)

Plot to check the distribution of the local scores:

In [ ]:
from liana.method.sp._spatialdm import spatialdm

In [ ]:
spatialdm(adata=adata, resource_name="MouseConsensus", pvalue_method='analytical', verbose=True, use_raw=False)

In [ ]:
local_pvals = li.ut.obsm_to_adata(adata, obsm_key='local_pvals')

In [ ]:
local_r = li.ut.obsm_to_adata(adata, obsm_key='local_r')

In [ ]:
sc.pl.spatial(local_pvals, color=['Sema4d&Plxnb3'])

In [ ]:
sc.pl.spatial(local_r, color=['Sema4d&Plxnb3'])

In [ ]:
adata.obsm['local_r']

In [ ]:
def embed_score_to_adata(adata, score, score_name):
    score =  pd.DataFrame(score,
                          columns=adata.obsm['local_r'].columns,
                          index=adata.obsm['local_r'].index)
    adata.obsm[score_name] = score

In [ ]:
# Masked Pearson correlation
embed_score_to_adata(adata, masked_pc, 'masked_pc')

# Vectorized Pearson correlation
embed_score_to_adata(adata, local_pc.T, 'local_pc')

In [ ]:
# Masked Spearman correlation
embed_score_to_adata(adata, masked_sp, 'masked_sp')
# Vectorized Spearman correlation
embed_score_to_adata(adata, local_sp.T, 'local_sp')

In [ ]:
# Vectorized Jaccard index
embed_score_to_adata(adata, local_jc.T, 'local_jc')

In [ ]:
# Vectorized Cosine similarity
embed_score_to_adata(adata, local_cos.T, 'local_cos')

Plot to check the distribution of the local scores:

In [ ]:
sc.pl.spatial(adata, color=['Sema4d', 'Plxnb3'], use_raw=False)

In [ ]:
local_r = li.ut.obsm_to_adata(adata, obsm_key='local_r')
sc.pl.spatial(local_r, color=['Sema4d&Plxnb3'])

In [ ]:
sc.pl.spatial(local_pvals, color=['Sema4d&Plxnb3'], cmap='viridis_r')

Pearson

In [ ]:
masked_pc = li.ut.obsm_to_adata(adata, obsm_key='masked_pc')
sc.pl.spatial(masked_pc, color=['Sema4d&Plxnb3'])

In [ ]:
np.average(masked_pc.X)

In [ ]:
local_pc = li.ut.obsm_to_adata(adata, obsm_key='local_pc')
sc.pl.spatial(local_pc, color=['Sema4d&Plxnb3'])

In [ ]:
np.average(local_pc.X)

In [ ]:
local_sp = li.ut.obsm_to_adata(adata, obsm_key='local_sp')
sc.pl.spatial(local_sp, color=['Sema4d&Plxnb3'])

In [ ]:
masked_sp = li.ut.obsm_to_adata(adata, obsm_key='masked_sp')
sc.pl.spatial(masked_sp, color=['Sema4d&Plxnb3'])

Jaccard

In [ ]:
local_jc = li.ut.obsm_to_adata(adata, obsm_key='local_jc')
sc.pl.spatial(local_jc, color=['Sema4d&Plxnb3'])

Cosine

In [ ]:
local_cos = li.ut.obsm_to_adata(adata, obsm_key='local_cos')
sc.pl.spatial(local_cos, color=['Sema4d&Plxnb3'])

Pvals

In [ ]:
embed_score_to_adata(adata, local_pc_pvals.T, 'local_pc_pvals')
local_pc_pvals = li.ut.obsm_to_adata(adata, obsm_key='local_pc_pvals')

In [ ]:
sc.pl.spatial(local_pc_pvals, color=['Sema4d&Plxnb3'], cmap='viridis_r')

In [ ]:
# LASSO regression on each spot?